In [1]:
import pandas as pd
import json
import numpy as np
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_json('dwts_scraper/scores-new.json')
df['performance_id'] = np.arange(df.shape[0])

df = df.replace(r'^\s*$', np.nan, regex=True)

df.drop( df.query(" couple== ['Dance-off','Dance Duel'] ").index, inplace = True)
df.rename(columns = {'judge': 'judge_excluded', "dance": "dance_style"}, inplace = True)

df.dropna(subset=['couple'], inplace=True)

df.columns.sort_values()

Index(['broadway_show', 'cher_music', 'cirque_du_soleil_show',
       'classical_music', 'couple', 'dance_&_theme', 'dance_chosen_by',
       'dance_style', 'disney_film', 'dynamic_duo', 'elvis_music', 'era',
       'film', 'film_series', 'film_theme', 'genre', 'guest_judge',
       'horror_film_show', 'icon', 'icons', 'james_bond_film',
       'judge_excluded', 'judge_phrase', 'mentor', 'michael_buble_music',
       'motown_music', 'music', 'music_by_britney_spears',
       'music_by_janet_jackson', 'music_by_queen', 'music_from_grease',
       'musical', 'original_couple', 'performance_id', 'performance_scores',
       'professionals', 'result', 'scores', 'season', 'stevie_wonder_music',
       'team_captain', 'technical_scores', 'television_show',
       'trio_dance_partner', 'trio_partner', 'tv_show', 'villain',
       'week_title'],
      dtype='object')

In [3]:
df['judge'] = df['judge_phrase'].str.strip(to_strip=" .\n").str.split(", ", expand = False)

In [4]:
# week_cols = df['week_title'].str.extractall(r"Week (?P<week>\d+):?\s?(?P<week_theme>.*$)")

df['week'] = df['week_title'].str.extract(r'Week (\d+)')
df['week_theme'] = df['week_title'].str.extract(r': (.*?)$') #NaN if no Week Theme.
df['week_theme'].fillna('No Theme', inplace = True) # replace NaNs

In [5]:
# Handle 
# "Caveman" Hustle 
# Argentine tango

df['michael_jackson_theme'] = df['dance_&_theme'].str.extract(r'"(.*?)"')
df['michael_jackson_dance'] = df['dance_&_theme'].str.extract(r'(?:".*?")?(.*?)$')
df['dance_style'] = np.where(df['dance_&_theme'].notna(), df['michael_jackson_dance'], df['dance_style'])
df.drop(columns = ['michael_jackson_dance'], inplace = True)

# df[df['dance_&_theme'].notna()]

In [6]:
df['dance_styles'] = df['dance_style'].str.split("---")

df['primary_dance_style'] = df.dance_styles.apply(lambda x: x[0])
df['primary_dance_style'] = df['primary_dance_style'].str.replace("&",'').str.strip().str.title()

In [7]:
df[df['era'].notna()]

,couple,scores,dance_style,music,result,season,week_title,judge_phrase,technical_scores,performance_scores,film,era,judge_excluded,original_couple,dance_chosen_by,disney_film,film_theme,elvis_music,trio_partner,professionals,james_bond_film,music_by_britney_spears,stevie_wonder_music,dance_&_theme,dynamic_duo,motown_music,film_series,icons,cher_music,classical_music,mentor,music_from_grease,tv_show,horror_film_show,michael_buble_music,trio_dance_partner,music_by_queen,villain,cirque_du_soleil_show,music_by_janet_jackson,genre,icon,broadway_show,television_show,musical,team_captain,guest_judge,performance_id,judge,week,week_theme,michael_jackson_theme,dance_styles,primary_dance_style
206,Chad & Cheryl,"24 (8, 8, 8)",Jive,"""Love Man""—Otis Redding",Safe,10,Week 8: Race to the Semifinals,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.\n",NaN,NaN,NaN,1960s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206,"[Carrie Ann Inaba, Len Goodman, Bruno Tonioli]",8,Race to the Semifinals,NaN,[Jive],Jive
208,Niecy & Louis,"20 (7, 6, 7)",Paso doble,"""Rhythm is a Dancer""—Snap!",Eliminated,10,Week 8: Race to the Semifinals,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.\n",NaN,NaN,NaN,1990s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208,"[Carrie Ann Inaba, Len Goodman, Bruno Tonioli]",8,Race to the Semifinals,NaN,[Paso doble],Paso Doble
211,Erin & Maks,"25 (8, 8, 9)",Rumba,"""Missing You""—John Waite",Safe,10,Week 8: Race to the Semifinals,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.\n",NaN,NaN,NaN,1980s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211,"[Carrie Ann Inaba, Len Goodman, Bruno Tonioli]",8,Race to the Semifinals,NaN,[Rumba],Rumba
213,Evan & Anna,"26 (9, 8, 9)",Cha-cha-cha,"""Bulletproof""—La Roux",Safe,10,Week 8: Race to the Semifinals,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.\n",NaN,NaN,NaN,Futuristic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,213,"[Carrie Ann Inaba, Len Goodman, Bruno Tonioli]",8,Race to the Semifinals,NaN,[Cha-cha-cha],Cha-Cha-Cha
219,Nicole & Derek,"30 (10, 10, 10)",Paso doble,"""Spanish Guitar""—Bo Diddley",Safe,10,Week 8: Race to the Semifinals,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.\n",NaN,NaN,NaN,1950s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,219,"[Carrie Ann Inaba, Len Goodman, Bruno Tonioli]",8,Race to the Semifinals,NaN,[Paso doble],Paso Doble
481,Mýa & Dmitry,"30 (10, 10, 10)",Samba,"""Bad Girls""—Donna Summer",Safe,9,Week 8,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.\n",NaN,NaN,NaN,'70s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,481,"[Carrie Ann Inaba, Len Goodman, Bruno Tonioli]",8,No Theme,NaN,[Samba],Samba
483,Aaron & Karina,"27 (9, 9, 9)",Samba,"""Two Princes""—Spin Doctors",Eliminated,9,Week 8,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.\n",NaN,NaN,NaN,'90s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,483,"[Carrie Ann Inaba, Len Goodman, Bruno Tonioli]",8,No Theme,NaN,[Samba],Samba
496,Joanna & Derek,"29 (9, 10, 10)",Paso doble,"""Living on Video""—Trans-X",Safe,9,Week 8,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.\n",NaN,NaN,NaN,Futuristic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,496,"[Carrie Ann Inaba, Len Goodman, Bruno Tonioli]",8,No Theme,NaN,[Paso doble],Paso Doble
498,Kelly & Louis,"26 (8, 9, 9)",Jive,"""River 

In [8]:
df[df['primary_dance_style'].str.contains(r'\d+s', regex= True, na=False)]

,couple,scores,dance_style,music,result,season,week_title,judge_phrase,technical_scores,performance_scores,film,era,judge_excluded,original_couple,dance_chosen_by,disney_film,film_theme,elvis_music,trio_partner,professionals,james_bond_film,music_by_britney_spears,stevie_wonder_music,dance_&_theme,dynamic_duo,motown_music,film_series,icons,cher_music,classical_music,mentor,music_from_grease,tv_show,horror_film_show,michael_buble_music,trio_dance_partner,music_by_queen,villain,cirque_du_soleil_show,music_by_janet_jackson,genre,icon,broadway_show,television_show,musical,team_captain,guest_judge,performance_id,judge,week,week_theme,michael_jackson_theme,dance_styles,primary_dance_style


In [9]:
# coalesce 


music_cols = ['elvis_music', 'classical_music', 'motown_music', 'stevie_wonder_music', 
              'music_by_britney_spears', 'music_from_grease', 'cher_music',
              'music_by_queen', 'music_by_janet_jackson', 'michael_buble_music']

# some have the music in songs
# df['music'] = df['music'].fillna(df['songs'])


df['music'] = np.where(df['elvis_music'].notna(), df['elvis_music'] + "—Elvis Presley", df['music'])
df['music'] = np.where(df['classical_music'].notna(), df['classical_music'], df['music'])
df['music'] = np.where(df['motown_music'].notna(), df['motown_music'], df['music'])
df['music'] = np.where(df['stevie_wonder_music'].notna(), df['stevie_wonder_music'] + "—Stevie Wonder", df['music'])
df['music'] = np.where(df['music_by_britney_spears'].notna(), df['music_by_britney_spears'] + "—Brittany Spears", df['music'])
df['music'] = np.where(df['music_from_grease'].notna(), df['music_from_grease'], df['music'])
df['music'] = np.where(df['cher_music'].notna(), df['cher_music'] + "—Cher", df['music'])
df['music'] = np.where(df['music_by_queen'].notna(), df['music_by_queen'] + "—Queen", df['music'])
df['music'] = np.where(df['music_by_janet_jackson'].notna(), df['music_by_janet_jackson'] + "—Janet Jackson", df['music'])
df['music'] = np.where(df['michael_buble_music'].notna(), df['michael_buble_music'] + "—Michael Buble", df['music'])




In [10]:
# df[df['music_from_grease'].notna()]
df.drop(columns = music_cols, inplace = True)

In [11]:

df['songs'] = df['music'].str.split("---")
df['primary_song_phrase'] = df.songs.apply(lambda x: x[0])
df[['primary_song','primary_song_artist']] = df.primary_song_phrase.str.extract(r'(.*)[—―一]+(.*?)$')

# df['primary_song_artist_norm'] = df['primary_song_artist'].str.lower()

In [12]:
#df.head(500)[['primary_song','primary_artist']]
df[df['primary_song'].isna()]

# drop empty scores.  Some sort of extra line in the table at https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_13)#Week_10:_Finals

,couple,scores,dance_style,music,result,season,week_title,judge_phrase,technical_scores,performance_scores,film,era,judge_excluded,original_couple,dance_chosen_by,disney_film,film_theme,trio_partner,professionals,james_bond_film,dance_&_theme,dynamic_duo,film_series,icons,mentor,tv_show,horror_film_show,trio_dance_partner,villain,cirque_du_soleil_show,genre,icon,broadway_show,television_show,musical,team_captain,guest_judge,performance_id,judge,week,week_theme,michael_jackson_theme,dance_styles,primary_dance_style,songs,primary_song_phrase,primary_song,primary_song_artist


In [13]:
# Some weeks (e.g., random have a `score` column instead of a `scores` column.  Rename that.
# this seems to have been fixed on wikipedia.

# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_20)#Weekly_scores Week 5.
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_17)#Weekly_scores Week 6.


# https://stackoverflow.com/questions/34989341/how-to-remove-nan-value-while-combining-two-column-in-panda-data-frame
# df['scores'] = df['scores'].fillna(df['score'])

In [14]:
df.columns.sort_values()

Index(['broadway_show', 'cirque_du_soleil_show', 'couple', 'dance_&_theme',
       'dance_chosen_by', 'dance_style', 'dance_styles', 'disney_film',
       'dynamic_duo', 'era', 'film', 'film_series', 'film_theme', 'genre',
       'guest_judge', 'horror_film_show', 'icon', 'icons', 'james_bond_film',
       'judge', 'judge_excluded', 'judge_phrase', 'mentor',
       'michael_jackson_theme', 'music', 'musical', 'original_couple',
       'performance_id', 'performance_scores', 'primary_dance_style',
       'primary_song', 'primary_song_artist', 'primary_song_phrase',
       'professionals', 'result', 'scores', 'season', 'songs', 'team_captain',
       'technical_scores', 'television_show', 'trio_dance_partner',
       'trio_partner', 'tv_show', 'villain', 'week', 'week_theme',
       'week_title'],
      dtype='object')

In [15]:
# df[df['season'] == '']

In [16]:
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_9)#Weekly_scores Week 6 has a 'ranked order' score dance.
# Dance Offs are also a problem, some are separate tables, some are bottom rows in tables.






In [17]:
# Some weeks have a technical score/performance score. but no scores column.

# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_11)#Weekly_scores Week 4
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_10)#Weekly_scores Week 4

# double_score = df[df['technical_score'].notna()]

# double_score[double_score.columns[~double_score.isnull().any()]]

In [18]:
# Quite a few rows from tables that shouldn't be parsed (after end of week shows).  e.g. tributes.
# Either fix in parsing or drop here.

# df.dropna(subset=['dance_style'], inplace=True)

# df[df['dance'].isna()]



In [19]:
# season 22, week 8 has a very difficult judges score arrangement
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_22)#Week_8:_Judges'_Team-up_Challenge

#  total (9, X, 9, 9) where the final score is "america's score" and the X is the score that would have been the score from the judge
# in df['judge'] column.  gah.  but those are multi couple dances.  phew.

In [20]:
# editing on the pages made this moot.
# Some couple columns have additional info which is in parens in the column. This creates an NA couple column.

# pattern = r'(?P<couple_alone>.*?)\((?P<additional>.*?)\)$'

# # e.g., https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_27) Week 4.  Couple (Trio Dance Partner)
# df[['couple_alone','trio_partner']] = df['couple_trio_dance_partner_'].str.extract(pattern)
# df['couple'] = df['couple'].fillna(df['couple_alone'])
# df.drop(columns='couple_alone', inplace=True)

# # https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_16) Week 5 (Professionals).
# df[['couple_alone','additional_dancers']] = df['couple_professionals_'].str.extract(pattern)
# df['couple'] = df['couple'].fillna(df['couple_alone'])
# df.drop(columns='couple_alone', inplace=True)

# # couple_team_up_judge_ from https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_22)#Week_8:_Judges'_Team-up_Challenge
# # Some of These are multi-couple dances, with a judge listed i the couple column, and then an X for the score that the judge that coached the team would have given.
# # but others are regular dances (just a different column name)
# df['couple'] = df['couple'].fillna(df['couple_team_up_judge_'])

# # couple_judge_  from https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_11)#Week_10:_Finals is much simpler (just a judge chosen dance).
# df[['couple_alone','judge_choosing_dance']] = df['couple_judge_'].str.extract(pattern)
# df['couple'] = df['couple'].fillna(df['couple_alone'])
# df.drop(columns='couple_alone', inplace=True)

# # One remains from the couple_judge_ due to a substitution which then didn't have a judge (so the regex above doesn't catch it)
# # 2341 where the couple_judge_ is Nastia & Sasha[a]
# df['couple'] = df['couple'].fillna(df['couple_judge_'])
# # df.iloc[2341]

# df.drop(columns=['couple_trio_dance_partner_', 'couple_professionals_', 'couple_judge_'], inplace=True)

# df[df['couple'].isna()]

In [21]:
# Some have technical_score and performance_score.
# add a score_type column.
# split these out, going to be two rows, each with scores column and a score_type column.

double_scores = ( df[df['technical_scores'].notna()]
                  .drop(columns="scores")
                  .dropna(axis = 1, how = "all") )

val_cols = ['technical_scores','performance_scores']
id_cols = [ele for ele in double_scores if ele not in val_cols]
# now get two rows per
double_scores = double_scores.melt(id_vars = id_cols, value_vars = val_cols, value_name = "scores", var_name = "score_type")

single_scores = df.drop(df.index[df['technical_scores'].notna()])
# make others be score_type
single_scores['score_type'] = "single_score"
single_scores.drop(columns=['technical_scores','performance_scores'], inplace=True)

df_score_type = pd.concat([double_scores, single_scores])


In [22]:
# df_score_type.head()

In [23]:
df_score_type[['total_score','indiv_scores']] = df_score_type['scores'].str.extract(r'(?P<total_score>\d+)\s\((?P<indiv_scores>.*)\)')

# new_cols['judge_score_list'] = new_cols['indiv_scores'].str.split(",")


# df_new = pd.concat([df, new_cols], axis = 1)

In [24]:
# song_cols = df['music'].str.extract(r'"(?P<song_name>.*)"—(?P<song_artist>.*)$')
# song_cols
# df["music"].str.split(" / ") Some music columns have multiples.

In [25]:
# most are multi couple dances, with ranking scores.

df_score_type["couple_list"] = df_score_type["couple"].str.split("---")
df_score_type["couple_count"] = df_score_type["couple_list"].apply(len)



In [26]:
# Now eliminate multi-couple dances and dances with NA total scores (mostly dance offs and others). 
#  Might still be one or two issues with total scores but not individual scores.

df_single_couple = df_score_type.loc[(df_score_type['couple_count'] == 1) & df_score_type['total_score'].notna() ].copy()

# df_single_couple.head()



In [27]:
# df_single_couple[df_single_couple['indiv_scores'].isna()]

In [28]:
# df_single_couple.head()

df_single_couple['judge_score_list'] = df_single_couple['indiv_scores'].str.split(", ")

# df_single_couple['judge_score_list']
df_single_couple['judge_score_count'] = df_single_couple['judge_score_list'].apply(len)

df_single_couple['judge_count'] = df_single_couple['judge'].apply(len)


# a few have a total score, but not an individual score.  e.g., s9w10.

In [29]:
#pd.set_option('max_colwidth', None) 
#df_single_couple[df_single_couple['season'] == 31][['week','judge_phrase']]

In [30]:
df_single_couple.loc[df_single_couple['judge_score_count'] != df_single_couple['judge_count']]

# Trouble in 
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_23)#Week_5:_Most_Memorable_Year_Night
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_31)#Week_6:_Michael_Bubl%C3%A9_Night

,couple,dance_style,music,result,season,week_title,judge_phrase,performance_id,judge,week,week_theme,dance_styles,primary_dance_style,songs,primary_song_phrase,primary_song,primary_song_artist,score_type,scores,film,era,judge_excluded,original_couple,dance_chosen_by,disney_film,film_theme,trio_partner,professionals,james_bond_film,dance_&_theme,dynamic_duo,film_series,icons,mentor,tv_show,horror_film_show,trio_dance_partner,villain,cirque_du_soleil_show,genre,icon,broadway_show,television_show,musical,team_captain,guest_judge,michael_jackson_theme,total_score,indiv_scores,couple_list,couple_count,judge_score_list,judge_score_count,judge_count


In [31]:
df_final = ( df_single_couple
              #  .head(5)
               .explode(['judge','judge_score_list'])
               .rename(columns={'judge_score_list': 'judge_score'})
               .drop(columns=['indiv_scores','judge_phrase','scores'])
               .assign(judge = lambda df: df.judge.str.strip(" ."))
)

df_final.drop( df_final.query(" judge_score == 'X' ").index, inplace = True)

df_final['judge_score_id'] = np.arange(df_final.shape[0])

In [32]:
# rename "Guest judge" with value from df_final['guest_judge']
df_final['judge'] = np.where(df_final['judge'] == 'Guest judge', df_final['guest_judge'], df_final['judge'])

# fix Julianne Hough (Night 1 only)
df_final['judge'] = np.where(df_final['judge'] == 'Julianne Hough (Night 1 only)', 'Julianne Hough', df_final['judge'])

# sorted(df_final['judge'].unique().tolist())

In [33]:
# df_final[df_final['judge_score'] == '2']

In [34]:
# sorted(df_final['judge_score'].unique().tolist())

In [35]:
# df_final[df_final['judge_score'] == 'X']



In [36]:
# Now need to group_by and average scores to get rid of the technical_scores and performance_scores.

In [37]:
all_cols = df_final.columns.values.tolist()

to_remove = ['judge_excluded', 'couple_list', 'couple_count', 'judge_score_count', 
             'judge_count', 'dance_styles', 'songs', 'primary_song_phrase', 'dance_&_theme']

all_cols = [ele for ele in all_cols if ele not in to_remove]

front_cols = [ 'judge_score_id', 'judge','judge_score', 'score_type', 'performance_id','season','week','week_theme', 'couple',
              'primary_dance_style','primary_song', 'primary_song_artist', 'total_score' ]

remaining_cols = [ele for ele in all_cols if ele not in front_cols]

reordered = front_cols + remaining_cols

In [38]:
df_final = df_final[reordered]

In [39]:
df_final.to_csv("dwts_dataset.csv", index = False)

In [44]:
df_final.shape

(9787, 48)

# Produce tables

```
performances (id, song_id, dance_style_id, couple_id, season_id, week_id, week_theme_id)
dance_styles (id, name) 
songs (id, name, artist_id)
artists (id, name)
scores (id, performance_id, person_id, score)
seasons (id, season_no, year_broadcast)
week_themes (id, name, season_id)
              
couples (id, name, celebrity_person_id, pro_person_id) # maybe.

people # Note that some work needed to match up couples to people.
```

In [41]:
# dance_styles_series = df_final['dance_style'].unique()
# dance_styles = pd.DataFrame({'id': np.arange(1, len(dance_styles_series) + 1), 'name': dance_styles_series})
# dance_styles.to_csv("dwts_dataset/dance_styles.csv", index = False)

In [42]:
# couples_series = df_final['couple'].unique()
# couples = pd.DataFrame({'id': np.arange(1, len(couples_series) + 1), 'name': couples_series})
# couples.to_csv("dwts_dataset/couples.csv", index = False)

In [46]:
for dim in ['couple','primary_dance_style', 'primary_song', 'week_theme', 'primary_song_artist', 'judge']:
    df_final[dim + "_id"] = df_final[dim].factorize()[0] + 1
    filename = "dwts_dataset/" + dim + "s.csv"
    # os.remove(filename)
    df_final[[dim + "_id", dim]].drop_duplicates().to_csv(filename, index = False)

# df_final['couple_id'] = df_final.couple.factorize()[0] + 1
# df_final[['couple_id','couple']].drop_duplicates().to_csv("dwts_dataset/couples.csv", index = False)

In [57]:
performances = df_final[['performance_id', 'primary_song_id', 'primary_dance_style_id', 'couple_id', 'season', 'week', 'week_theme_id']]
performances.drop_duplicates().to_csv("dwts_dataset/performances.csv", index = False)

scores = df_final[['performance_id', 'judge_id', 'score_type', 'judge_score']].copy()
# scores.drop_duplicates(inplace = True)

scores['score_id'] = np.arange(1, len(scores) + 1)
scores = scores[['score_id', 'performance_id', 'judge_id', 'score_type', 'judge_score']]
scores.to_csv("dwts_dataset/scores.csv", index = False)

In [58]:
# scores['judge_score'].unique()

array(['6', '9', '8', '5', '7', '4', '10', '2', '3', '6.5', '7.5', '5.5',
       '8.5', '9.5'], dtype=object)

In [ ]:
# To do people.  Need judges from df_final['judge'].  Then need to split couples.